# Imports

In [13]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random
import math

# Set up constants

In [31]:
k_timber = .147
k_air = .0262

#For a pin oak, they can grow anywhere from 60'-70'.
#Their final spread can be anwywhere from 25'-40'

#Tree height
#60' -> 18.288m
#70' -> 21.336m
#standard deviation for height: (21.336 - 18.288)/2 = 1.524m
#Gaussian distribution

#Tree spread
#25' -> 7.62m
#40' -> 12.192m
#standard deviation for spread: (12.192 - 7.62)/2 = 2.286m
#Gaussian distribution

PIN_OAK_MIN_HT = 18.288 #m
PIN_OAK_HT_STD = 1.524 #m

PIN_OAK_MIN_SD = 7.62 #m
PIN_OAK_SD_STD = 2.286 #m

STAGE2_HORIZON_TEMP = 500  #°C
STAGE3_HORIZON_TEMP = 700  #°C
STAGE4_HORIZON_TEMP = 1000 #°C

"""
                    +---------------------+
                    |     ____________    | -+
                    |    /            \   |  |
                    |   |             |   |  |
                    |    \  | || |  / /   |  |                Height:
                    |     \ |/ | / / |    |  |     +---------------------------+
                    |      \ \ || / /     |  +-----|min = 18.288m, std = 1.524m|
                    |       |      |      |  |     +---------------------------+
                    |       |      |      |  |
                    |       |      |      |  |
                    |       |      |      |  |
                    |       |      |      |  |
                    |______/________\_____| -+
                    +---------------------+
                         |            |
                         +-----+------+
                               |      Spread:
                            +-------------------------+   
                            |min = 7.62m, std = 2.286m|
                            +-------------------------+
""";

In [137]:
class FireBoard:
    class Cell:
        def __init__(self, x, y, height=None, temp=None):
            self.state = 0
            self.x = x
            self.y = y

            if height is None:
                self.height = PIN_OAK_HT_STD*random.random()+PIN_OAK_AVG_HT
                if temp is None:
                    self.temp = 1*random.random()+24.5
                else:
                    self.temp = temp
            else:
                self.height = height
                self.temp = 25;

        def update(self, board):
            if self.state == 0:
                #Find the highest state of the neighbours
                max_state = self.state
                #Check each neighbor:
                # Check upper levels
                if (self.x > 0):
                    for i in range(3):
                        if max_state < board[x-1][y+i].state
                            max_state = board[x-1][y+i].state
                        if max_state == 3:
                            break
                if max_state == 3:
                    self.state = 3
                    

        def burn(self):
            pass

    def random_cell(self, x, y, ht=None, chance=.65):
        if random.random() >= 1-chance:
            return self.Cell(x, y, height=ht)
        else:
            return self.Cell(x, y, height=0)

    def __init__(self, board=None, nrows=None, ncols=None, tree_chance=.65, wdir=0, wspd=1.0):
        self.rows = 20
        self.cols = 20

        if board is not None:
            self.board = board
            self.rows = len(board)
            self.cols = len(board[0])
        else:
            if (nrows is not None) and (nrows >= 3):
                self.rows = nrows
            if (ncols is not None) and (ncols >= 3):
                self.cols = ncols

            self.board = np.array(
                [[self.random_cell(i, j, chance=tree_chance) for j in range(self.cols)] for i in range(self.rows)],
                dtype="object"
            )

    def imshow_height(self):
        fig, ax = plt.subplots()
        im = ax.imshow([[self.board[i][j].height for j in range(self.cols)] for i in range(self.rows)], cmap="copper")
        ax.set_title("Cell Heights")
        ax.set_axis_off()
        fig.colorbar(im, ax=ax)
        return fig, ax

    def imshow_temp(self):
        fig, ax = plt.subplots()
        im = ax.imshow([[self.board[i][j].temp for j in range(self.cols)] for i in range(self.rows)], cmap="jet")
        ax.set_title("Cell Temperatures")
        ax.set_axis_off(np)
        fig.colorbar(im, ax=ax)
        return fig, ax

In [152]:
nrows=25
ncols=25

fb = FireBoard(nrows=nrows, ncols=ncols, tree_chance=.2)
# im = plt.imshow(
#     [[(fb.board[i][j].height > 0) for j in range(fb.cols)] for i in range(fb.rows)],
#     cmap="gray"
# )
# plt.title("Tree map")
# plt.colorbar(im, ticks=[0, 1])
# fig, ax = fb.imshow_height()
# fig, ax = fb.imshow_temp()
# plt.show()

[[[<__main__.FireBoard.Cell object at 0x000002E4B2AE4550>


# Create burning animation

In [6]:
%matplotlib notebook

tree_chance = .6

cells = [[cell(height=None) if random.random() >= (1-tree_chance) else cell(height=0) for j in range(20)] for i in range(20)]

plt.close()
fig, ax = plt.subplots()

def get_heights(data):
    rows = len(data)
    cols = len(data[0])
    return [[data[i][j].height for j in range(cols)] for i in range(rows)]

row = math.floor(random.random()*20)
col = math.floor(random.random()*20)

cells[row][col].height = 25

im = ax.imshow(get_heights(cells), cmap='viridis')
ax.set_xticks([])
ax.set_yticks([])

def animate(frame_num):
    cells[row][col].burn()
    im.set_data(get_heights(cells))
    im.autoscale()
    return [im]

anim = FuncAnimation(fig, animate, frames=10, interval=1000)
anim.save("tree.mp4")
plt.show()
print("Done.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cmaps = [('Perceptually Uniform Sequential', [
            'viridis', 'plasma', 'inferno', 'magma', 'cividis']),
         ('Sequential', [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
         ('Sequential (2)', [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']),
         ('Diverging', [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']),
         ('Cyclic', ['twilight', 'twilight_shifted', 'hsv']),
         ('Qualitative', [
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c']),
         ('Miscellaneous', [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
            'gist_ncar'])]

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(f"{cmap_category} colormaps", fontsize=14)

    for ax, cmap_name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=cmap_name)
        ax.text(-.01, .5, cmap_name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()


for cmap_category, cmap_list in cmaps:
    plot_color_gradients(cmap_category, cmap_list)